# 1. HOTRG

In [ ]:
import argparse
import numpy as np

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_name")
    parser.add_argument("--symm", help="keep symmetric or not")
    parser.add_argument("--Tmin")
    parser.add_argument("--Tmax")
    parser.add_argument("--Tstep")
    parser.add_argument("--dcut")
    parser.add_argument("-US", "--unit_size")
    parser.add_argument("--trgstep")

    args = parser.parse_args()

    print("{}, symm={}, D={}".format(args.model_name, args.symm, args.dcut))
    Ts = np.arange(args.Tmin, args.Tmax+args.Tstep, args.Tstep)
    print("T = ", Ts)

    Ls = args.unit_size * 2**np.arange(0, args.trgstep+1)
    print("L = ", Ls)

if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] [--model_name MODEL_NAME] [--symm SYMM]
                             [--Tmin TMIN] [--Tmax TMAX] [--dcut DCUT]
                             [-US UNIT_SIZE] [--trgstep TRGSTEP]
ipykernel_launcher.py: error: unrecognized arguments: --f=/home/chansh/.local/share/jupyter/runtime/kernel-v3f1642175bf57149807d0f054222425f1412adafc.json


SystemExit: 2

/home/chansh/miniconda3/envs/cytnx_test2/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#a Parameters settings
model_name = "2D_M=3_XY" #"#model_name#"
symm = True #symm#
get_momentum = 1#get_momentum#

print("cytnx version: ", cy.__version__)

model = Model(model_name, symm)

HOTRG = Hotrg()
HOTRG.dcut = #D_HOTRG#
unit_size = #unit_size#
Lmax = #Lmax#
HOTRG.trgstep = int(np.floor(np.log2(Lmax/unit_size)))
Ls = unit_size * 2**np.arange(0, HOTRG.trgstep+1)
print("L = ", Ls)

# Folders creation
prefix = f"{model.name}{'_symm' if model.symm else ''}/"
Folder_T0    = prefix + f"T0_tensor_{float_type}/"
Folder_Norms = prefix + f"Norms_{float_type}/"
Folder_P0    = prefix + f"P0_tensor_{float_type}/"
Folder_Uy    = prefix + f"Uy_tensor_{float_type}/"

for Folder in [Folder_T0, Folder_P0, Folder_Norms, Folder_Uy]:
    makeFolder(Folder)

iteration = #iter#
if iteration == 300:
    Ts_tot = [model.Tc]
elif iteration == 301:
    Ts_tot = np.arange(-10,10+1,2) * 4e-3 + model.Tc
    Ts_tot = Ts_tot[Ts_tot != model.Tc]
else:
    Folder_Ts = prefix + "T_list_iter/"
    Ts_tot = pd.read_csv(Folder_Ts + "T_iter{}_D{}_n{}_US{}.csv".format(iteration, HOTRG.dcut, n, unit_size))['T'].values

# Ts = Ts_Tot
iT = #iT#
Ts = [Ts_tot[iT]]

for T in Ts:
    for L in Ls:
        print("L={}".format(L))
        if L == unit_size:
            T0 = model.T0_1x1(T)
            T0 = ExactCon.contract_LxL([T0]*unit_size**2, unit_size)
            Norm = max([block.Abs().Max().item() for block in T0.get_blocks()])
            T0 /= Norm
            T0 = T0.astype(dtype_cy)
            if get_momentum:
                P0 = model.P0_1x1()
                P0 = ExactCon.contract_vertical([P0]*unit_size)
            
            Norms = np.empty(0, dtype=float_type)
            if float_type == "float32":
                Norms = np.append(Norms, np.float32(Norm))
            if float_type == "float64":
                Norms = np.append(Norms, Norm)
            print(T0.dtype_str(), P0.dtype_str(), Norms.dtype)
        else:
            T0, Uy, Ny = HOTRG.update_pure(T0, T0, "y", HOTRG.dcut, "Abs_Max")
            if get_momentum:
                P0, _ = HOTRG.update_impurity(P0, P0, "y", Uy, 1)
            Uy.Save(Folder_Uy + "Uy_T{}_L{}_D{}_US{}_{}".format(T,L,HOTRG.dcut,unit_size,model.name))
            del Uy, _

            print(T0.dtype_str(), P0.dtype_str(), Norms.dtype)
        
            T0, Ux, Nx = HOTRG.update_pure(T0, T0, "x", HOTRG.dcut, "Abs_Max")
            if float_type == "float32":
                Norms = np.append(Norms, np.float32(Ny**2 * Nx))
            if float_type == "float64":
                Norms = np.append(Norms, Ny**2 * Nx)
            del Ux, Ny, Nx

            print(T0.dtype_str(), P0.dtype_str())
            
        T0.Save(Folder_T0 + "T0_T{}_L{}_D{}_US{}_{}".format(T,L,HOTRG.dcut,unit_size,model.name))
        if get_momentum:
            P0.Save(Folder_P0 + "P0_T{}_L{}_D{}_US{}_{}".format(T,L,HOTRG.dcut,unit_size,model.name))
        np.save(Folder_Norms + "Norms_T{}_L{}_D{}_US{}_{}.npy".format(T,L,HOTRG.dcut,unit_size,model.name), Norms)
